In [5]:
import os
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix
import json

outputfolder = "../out/AllonKleinLab/Experiment3"
inputfile = "../data/AllonKleinLab/Experiment3/stateFate_cytokinePerturbation_normed_counts.mtx"

HVF = True
TEST = False 
TEST_genes = 800
TEST_samples = 10000

k = 15

 # Load Dataset

In [6]:
se = csr_matrix(mmread( inputfile))
se = se[se.getnnz(1) > 0][:, se.getnnz(0) > 0]

if TEST:
    tgenes = min(TEST_genes, se.shape[1])
    tsamples = min(TEST_samples, se.shape[0])
    se.resize((tsamples, tgenes))
    se = se[se.getnnz(1)>0][:,se.getnnz(0)>0]

adata = sc.AnnData(se)
#sc.pp.normalize_total(adata, target_sum=1e6)
##sc.pp.log1p(adata)
sc.pp.scale(adata, zero_center=False)
sc.pp.highly_variable_genes(adata)

if HVF:
    adata = adata[:, adata.var.highly_variable]
    adata = adata[adata.X.getnnz(1)>0][:, adata.X.getnnz(0)>0]
print(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

View of AnnData object with n_obs × n_vars = 0 × 0
    var: 'mean', 'std', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'


ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.

 # Visualize data

In [ ]:
# PCA plot
imgname = os.path.join(outputfolder, "PCA.png")
print(f"Saving Image --- {imgname}")
sc.pl.pca(adata, save=imgname.split('/')[-1])

In [ ]:
# UMAP plot
imgname = os.path.join(outputfolder, "UMAP.png")
print(f"Saving Image --- {imgname}")
sc.pl.umap(adata, save=imgname.split('/')[-1])

In [ ]:
# Elbow plot
imgname = os.path.join(outputfolder, "elbow.pdf")
print(f"Saving Image --- {imgname}")
sc.pl.pca_variance_ratio(adata, log=True, save=imgname.split('/')[-1])

In [ ]:
## Archetypes
#import archetypes as arch
#from time import time
#
#aa_kwargs = {
#    "n_archetypes": 4,
#    "n_init": 5,
#    "max_iter": 10000,
#    "verbose": True,
#    "tol": 1e-3,
#}
#
#mod0 = arch.AA(**aa_kwargs, algorithm_init="random")
#
#t0=time()
#mod0.fit_transform(adata.X.toarray())
#t1=time()
#
#print(f"mod0: {t1-t0:.2f} seconds|RSS: {mod0.rss:.2f}")